In [2]:
import sys
at_colab = "google.colab" in sys.modules
if 'google.colab' in sys.modules:
    import os
    from google.colab import files
    # just check if we already uploaded, may we restart the runtime and run all cells
    if not os.path.isfile('Data Set Feedcalculator.xlsx'):
        uploaded = files.upload()

Saving Data Set Feedcalculator.xlsx to Data Set Feedcalculator.xlsx


In [3]:
if at_colab:
    import shutil
    if not shutil.which("pyomo"):
        !pip install -q pyomo
        assert(shutil.which("pyomo"))

     |████████████████████████████████| 9.1 MB 4.9 MB/s 
     |████████████████████████████████| 49 kB 5.1 MB/s 


In [4]:
if at_colab:
    if not shutil.which('/usr/bin/glpsol'):
        !sudo apt install libglpk-dev python3.8-dev libgmp3-dev
        !apt-get install -y -qq glpk-utils
        assert(shutil.which('/usr/bin/glpsol'))

    if not shutil.which('/usr/bin/cbc'):
        !apt-get install -y -qq coinor-cbc
        assert(shutil.which('/usr/bin/cbc'))

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libamd2 libbtf1 libcamd2 libccolamd2 libcholmod3 libcolamd2 libcxsparse3
  libglpk40 libgmp-dev libgmpxx4ldbl libgraphblas1 libklu1 libldl2 libmetis5
  libpython3.8 libpython3.8-dev libpython3.8-minimal libpython3.8-stdlib
  librbio2 libspqr2 libsuitesparse-dev libsuitesparseconfig5 libumfpack5
  python3.8 python3.8-minimal
Suggested packages:
  libiodbc2-dev gmp-doc libgmp10-doc libmpfr-dev python3.8-venv python3.8-doc
  binfmt-support
The following NEW packages will be installed:
  libamd2 libbtf1 libcamd2 libccolamd2 libcholmod3 libcolamd2 libcxsparse3
  libglpk-dev libglpk40 libgmp-dev libgmp3-dev libgmpxx4ldbl libgraphblas1
  libklu1 libldl2 libmetis5 libpython3.8 libpython3.8-dev libpython3.8-minimal
  libpython3.8-stdlib librbio2 libspqr2 libsuitesparse-dev
  libsuitesparseconfig5 libumfpack5 python3.8 python3.8-dev python3.8-mini

In [5]:
import pyomo.environ as pyo
import pandas as pd
from io import StringIO

In [6]:
data = pd.read_excel('Data Set Feedcalculator.xlsx',sheet_name=None)

In [7]:
ingredients      = data['Ingredient Database']
nutrient_rules   = data['Nutrient Rules']
ingredient_rules = data['Ingredient Rules']
ingredient_rules

,Ingredient,LowerBound,UpperBound,Unnamed: 3,Combined Ingredient Rules,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,Unnamed: 10,Unnamed: 11,Unnamed: 12
0,barley,NaN,0.100,NaN,Upper Bound,0.17,0.35,0.06,0.3,0.25,0.07,0.08,0.25
1,blood,NaN,0.030,NaN,Ingredients,cotton,soybeanexp,rapecake,barley,maizebranhighq,fish,mbmeal,tapbran
2,boneash,NaN,0.030,NaN,NaN,sunflower,soybeanmeal,rapemeal,wheat,maizebranlowq,fishlq,mbmeal2,caswhole
3,mbmeal,NaN,0.050,NaN,NaN,gncake,soybeanmealhp,NaN,NaN,NaN,NaN,NaN,casfine
4,mbmeal2,NaN,0.080,NaN,NaN,copra,soybeanheat,NaN,NaN,NaN,NaN,NaN,cascoarse
5,cotton,NaN,0.060,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,copra,NaN,0.070,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,fats,NaN,0.050,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,fish,NaN,0.070,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,fishlq,NaN,0.070,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
available_ingredients = ingredients[ingredients.Availability][['Name','Reference name','Price']+list(nutrient_rules.Nutrient)].set_index('Reference name')
available_ingredients

,Name,Price,oebr,cp,cfibre,staew,ca,na,opp,dlysp,dmetp,dmcp,dthrp,dtryp,dvalp,dargp
Reference name,,,,,,,,,,,,,,,,
barley,Barley,0.26,2656.5,99.7,43.4,539.3,0.4,0.1,1.2,2.3,1.3,2.9,2.3,0.9,3.6,3.9
blood,Blood meal 80% CP,0.71,2450.0,800.0,10.0,0.0,0.5,5.8,1.3,51.3,6.9,13.8,25.4,8.7,49.6,24.8
boneash,Bone ash,0.18,0.0,0.0,0.0,0.0,289.3,9.5,82.9,0.0,0.0,0.0,0.0,0.0,0.0,0.0
cotton,Cotton seed cake,0.20,1730.0,362.9,169.8,33.6,2.2,0.0,3.1,8.8,3.9,8.1,8.0,3.2,11.8,32.7
fish,Fish meal 56% CP,0.63,2862.0,563.0,0.0,0.0,60.0,10.5,19.6,38.5,14.3,18.8,20.1,5.6,32.3,30.6
fishlq,Fish meal 40% CP,0.55,0.0,384.0,0.0,0.0,141.0,10.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
gnseeds,Groundnut seeds,0.59,5243.0,287.1,23.3,0.0,1.0,0.0,1.7,7.5,2.8,5.9,5.8,2.4,9.5,28.1
maize,Maize/ Corn,0.33,3227.8,76.3,20.8,648.5,0.1,0.0,0.7,1.4,1.4,2.7,2.1,0.4,2.9,3.1
maizebranhighq,"Maize bran, high quality",0.19,2893.4,90.6,39.6,489.3,1.3,0.2,1.4,1.8,1.6,3.1,2.4,0.5,3.8,3.9


In [9]:
ingredient_bounds = { ing : (lb if not pd.isna(lb) else 0,    # no lower bound translates into 0 as lower bound
                             ub if not pd.isna(ub) else None) # no upper bound becomes None
                      for ing,lb,ub in zip(ingredient_rules.Ingredient,ingredient_rules['LowerBound'],ingredient_rules['UpperBound']) }

In [10]:
ingredient_bounds_df = pd.DataFrame.from_dict(ingredient_bounds, orient='index')
ingredient_bounds_df = ingredient_bounds_df.set_axis(['LowerBound', 'UpperBound'], axis=1)
ingredient_bounds_df

,LowerBound,UpperBound
barley,0.0,0.100
blood,0.0,0.030
boneash,0.0,0.030
mbmeal,0.0,0.050
mbmeal2,0.0,0.080
cotton,0.0,0.060
copra,0.0,0.070
fats,0.0,0.050
fish,0.0,0.070
fishlq,0.0,0.070


In [11]:
ingredients_plus_bounds = pd.merge(left=available_ingredients, right=ingredient_bounds_df, left_on='Reference name', right_on=ingredient_bounds_df.index)
ingredients_plus_bounds.set_index('Reference name')

,Name,Price,oebr,cp,cfibre,staew,ca,na,opp,dlysp,dmetp,dmcp,dthrp,dtryp,dvalp,dargp,LowerBound,UpperBound
Reference name,,,,,,,,,,,,,,,,,,
barley,Barley,0.26,2656.5,99.7,43.4,539.3,0.4,0.1,1.2,2.3,1.3,2.9,2.3,0.9,3.6,3.9,0.0,0.100
blood,Blood meal 80% CP,0.71,2450.0,800.0,10.0,0.0,0.5,5.8,1.3,51.3,6.9,13.8,25.4,8.7,49.6,24.8,0.0,0.030
boneash,Bone ash,0.18,0.0,0.0,0.0,0.0,289.3,9.5,82.9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.030
cotton,Cotton seed cake,0.20,1730.0,362.9,169.8,33.6,2.2,0.0,3.1,8.8,3.9,8.1,8.0,3.2,11.8,32.7,0.0,0.060
fish,Fish meal 56% CP,0.63,2862.0,563.0,0.0,0.0,60.0,10.5,19.6,38.5,14.3,18.8,20.1,5.6,32.3,30.6,0.0,0.070
fishlq,Fish meal 40% CP,0.55,0.0,384.0,0.0,0.0,141.0,10.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.070
gnseeds,Groundnut seeds,0.59,5243.0,287.1,23.3,0.0,1.0,0.0,1.7,7.5,2.8,5.9,5.8,2.4,9.5,28.1,0.0,0.100
maize,Maize/ Corn,0.33,3227.8,76.3,20.8,648.5,0.1,0.0,0.7,1.4,1.4,2.7,2.1,0.4,2.9,3.1,0.2,NaN
maizebranhighq,"Maize bran, high quality",0.19,2893.4,90.6,39.6,489.3,1.3,0.2,1.4,1.8,1.6,3.1,2.4,0.5,3.8,3.9,0.0,0.250


In [ ]:
# nutrients = available_ingredients.columns[2:]
# nutrients

In [12]:
nutrient_bounds = { nut : (lb if not pd.isna(lb) else 0,     # no lower bound translates into 0 as lower bound
                           ub if not pd.isna(ub) else None)  # no upper bound becomes None
                    for nut,lb,ub in zip(nutrient_rules.Nutrient,nutrient_rules['Lower Bound'],nutrient_rules['Upper Bound']) 
                  }

In [13]:
nutrient_bounds_df = pd.DataFrame.from_dict(nutrient_bounds, orient='index')
nutrient_bounds_df = nutrient_bounds_df.set_axis(['LowerBound', 'UpperBound'], axis=1)
nutrient_bounds_df

,LowerBound,UpperBound
oebr,2750.0,2850.0
cp,155.0,195.0
cfibre,0.0,75.0
staew,300.0,NaN
ca,7.0,9.0
na,1.4,2.1
opp,3.5,NaN
dlysp,6.7,7.9
dmetp,2.8,NaN
dmcp,5.1,6.1


In [ ]:
# lst = [i for i in available_ingredients.index]
# lst

In [ ]:
# ingredient_bounds_updated = {}
# for k,v in ingredient_bounds.items():
#   for i in lst:
#     if i == k:
#       ingredient_bounds_updated[k]=v
# ingredient_bounds_updated

In [ ]:
#combined_ingredient_rules   = [] 
#set_of_availabe_ingredients = set(available_ingredients.index)
#for c in ['Unnamed: '+str(i) for i in range(5,13)]:
    #aux = ingredient_rules[[c]].dropna().values
    #aux = [ v[0] for v in aux ]
    #upperbound          = aux[0]
   # ingredients_in_rule = set(aux[1:]).intersection(set_of_availabe_ingredients)
    #if ingredients_in_rule:
        #combined_ingredient_rules.append((upperbound,ingredients_in_rule))
#combined_ingredient_rules

[(0.17, {'cotton', 'sunflower'}),
 (0.35, {'soybeanexp', 'soybeanmeal'}),
 (0.3, {'barley'}),
 (0.25, {'maizebranhighq', 'maizebranlowq'}),
 (0.07, {'fish', 'fishlq'}),
 (0.08, {'mbmeal'}),
 (0.25, {'casfine', 'caswhole', 'tapbran'})]

In [ ]:
#def CreateFeedCalculatorModel(ingredients_plus_bounds, nutrient_bounds_df):
 # m = pyo.ConcreteModel('Feed')

# m.ingredient = pyo.Set(initialize = ingredients_plus_bounds.index)
 # m.nutrient = pyo.Set(initialize = ['oebr','cp','cfibre','staew','ca', 'na', 'opp', 'dlysp', 'dmetp', 'dmcp', 'dthrp', 'dtryp', 'dvalp', 'dargp'])
  
 # return

In [68]:
def CreateFeedCalculatorModel(ingredients_plus_bounds, nutrient_bounds_df):

#( trophies=trophies, materials=materials ):
 model          = pyo.ConcreteModel('Feed Calculator')
 model.ingredients = pyo.Set( initialize = ingredients_plus_bounds.index )
 model.nutrients = pyo.Set( initialize = nutrient_bounds_df.index )

 model.n = pyo.Var( model.ingredients, within=pyo.NonNegativeReals )#если правильно понимаю, то создаю переменные: кол-во каждого ингридиента(вес)
#model.a_n = pyo.Var(model.nutrients, within=pyo.NonNegativeReals)#не знаю пригодиться ли эта фигня, скорее всего нет

 model.expenses = pyo.Objective(expr = pyo.quicksum( ingredients_plus_bounds.Price[i]*model.n[i] for i in model.n)
                          , sense=pyo.minimize)
 #не ебу как прописывать функции поэтому не работает, без скобок после return заработало
 def lower_bound_ing(model, i):
   return  model.n[i] >= ingredients_plus_bounds.LowerBound[i]
 model.lower_bound_ing = pyo.Constraint( model.ingredients, rule=lower_bound_ing)

#надо сделать функцию, а еще там 7, потому что я тупая и не знаю, что делать с Nan
 model.upper_bound_ing = pyo.ConstraintList()
 for i in range(7):
  model.upper_bound_ing.add(model.n[i]<=ingredients_plus_bounds.UpperBound[i])
#сумма всех ингридиентов по весу(вроде работает)
 def overall_weight_rule( model):
    return pyo.quicksum( model.n[i] for i in model.ingredients ) == 1
 model.overall_weight = pyo.Constraint( rule=overall_weight_rule )

 


#def upper_bound_ing(model,i): #что мы посылаем в функцию? объявление функций?
 #return (model.n[i] for i in model.ingredients <= ingredients_plus_bounds.UpperBound[i])#how to write in Python names of the columns with spaces 
#model.upper_bound_ing = pyo.Constraint(model.ingredients, rule=upper_bound_ing)
    
#def lower_bound_ing(model,i): 
 #eturn (model.n[i] for i in model.ingredients >= ingredients_plus_bounds.LowerBound[i])
#model.lower_bound_ing = pyo.Constraint(model.ingredients, rule=lower_bound_ing)

#def lower_bound_nut(model,i):
 #return pyo.quicksum(model.n[i]*model.a_n[j,i] for j in model.nutrients for i in model.ingredients >= nutrient_bounds_df.LowerBound[j]) #order of the leters in cycle for
#model.lower_bound_nut = pyo.Constraint(model.ingredients, rule=lower_bound_nut)

#def upper_bound_nut(model,i,j):
 #return pyo.quicksum(model.n[i]*model.a_n[j,i] for j in model.nutrients for i in model.ingredients <= nutrient_bounds_df.UpperBound[j]) #order of the leters in cycle for
#model.upper_bound_nut = pyo.Constraint(model.nutrients, rule=upper_bound_nut)
     
#model.pprint()
 return model

In [69]:
m = CreateFeedCalculatorModel(ingredients_plus_bounds, nutrient_bounds_df)
m.pprint()

3 Set Declarations
    ingredients : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   21 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20}
    nutrients : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   14 : {'oebr', 'cp', 'cfibre', 'staew', 'ca', 'na', 'opp', 'dlysp', 'dmetp', 'dmcp', 'dthrp', 'dtryp', 'dvalp', 'dargp'}
    upper_bound_ing_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    7 : {1, 2, 3, 4, 5, 6, 7}

1 Var Declarations
    n : Size=21, Index=ingredients
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :  None : False :  True : NonNegativeReals
          1 :     0 :  None :  None : False :  True : NonNegativeReals
          2 :     0 :  None :  None : False :  True : NonNegativeReals
      

In [36]:
model = pyo.ConcreteModel('Feed Calculator')

model.ingredients = pyo.Set( initialize = ingredients_plus_bounds.index )
model.nutrients = pyo.Set( initialize = nutrient_bounds_df.index )
model.n = pyo.Var( [i for i in model.ingredients], within=pyo.NonNegativeReals )#how to create the number of variables equal to a number of ingredients


model.expenses = pyo.Objective(expr = pyo.quicksum( ingredients_plus_bounds.Price[i]*model.n[i] for i in model.n)
                          , sense=pyo.minimize)

model.lower_bound_ing = pyo.ConstraintList()
for i in model.ingredients:
  model.lower_bound_ing.add(model.n[i]>=ingredients_plus_bounds.LowerBound[i])#rule for using bounds that equal to nan
#What to do with nan
model.upper_bound_ing = pyo.ConstraintList()
for i in range(7):
  model.upper_bound_ing.add(model.n[i]<=ingredients_plus_bounds.UpperBound[i])

model.overall_weight = pyo.ConstraintList()
model.overall_weight.add(pyo.quicksum([model.n[i] for i in model.ingredients]) == 1)


#def upper_bound_ing(model,i): #что мы посылаем в функцию? объявление функций?
 #return (model.n[i] for i in model.ingredients <= ingredients_plus_bounds.UpperBound[i])#how to write in Python names of the columns with spaces 
#model.upper_bound_ing = pyo.Constraint(model.ingredients, rule=upper_bound_ing)

#model.workers_per_day = pyo.ConstraintList()
#for d in model.days:
    #model.workers_per_day.add(pyo.quicksum([model.x[w + d]for w in model.workers]) == 14)

#model.min_per_worker = pyo.ConstraintList()
#for w in model.workers: 
    #model.min_per_worker.add(pyo.quicksum([model.x[w + d] for d in model.days]) >= min_hours['min'][w])

#model.max_per_day = pyo.ConstraintList()
#for w in model.workers:
    #for d in model.days:
       # model.max_per_day.add(model.x[w + d] <= workers[d][w])
        
model.pprint()

6 Set Declarations
    ingredients : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   21 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20}
    lower_bound_ing_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   21 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21}
    n_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   21 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20}
    nutrients : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   14 : {'oebr', 'cp', 'cfibre', 'staew', 'ca', 'na', 'opp', 'dlysp', 'dmetp', 'dmcp', 'dthrp', 'dtryp', 'dvalp', 'dargp'}
    overall_weight_index : Size=1, Index=None, Ordered=Insertion
   

In [ ]:
# def CreateFeedCalculatorModel( a_i ):
# m = pyo.ConcreteModel('Feed')
  
# m.name1 = pyo.Set(initialize = available_ingredients.index)
# m.nutrients = pyo.Set(initialize = nutrients)
# m.x = pyo.Var([w+d for w in m.name1 for d in m.nutrients], within=pyo.NonNegativeReals)

# m.cost = pyo.Objective(expr = pyo.quicksum(available_ingredients.Price[o]*m.name1[o] for o in m.name1), sense = pyo.minimize)

# m.max_amount = pyo.ConstraintList
# for w in m.name1:
#   m.max_amount.add(expr = pyo.quicksum([m.x[w+d] for d in m.nutrients]) == 1)

# m.lower_bound_nutrients = pyo.ConstraintList()
# for d in m.nutrients:
#   m.lower_bound_nutrients.add(pyo.quicksum([m.x[w+d]for w in m.name1]) >= nutrient_rules['Lower Bound'])

# m.upper_bound_nutrients = pyo.ConstraintList()
# for d in m.nutrients:
#   m.upper_bound_nutrients.add(pyo.quicksum([m.x[w+d]for w in m.name1]) <= nutrient_rules['upper Bound'])

# m.lower_bound_ingredients = pyo.ConstraintList()
# for w in m.name1:
#   for d in m.nutrients:
#     m.lower_bound_ingredients.add(m.x[w+d] >= ingredient_rules['Lower Bound'])

# m.upper_bound_ingredients = pyo.ConstraintList()
# for w in m.name1:
#   for d in m.nutrients:
#     m.upper_bound_ingredients.add(m.x[w+d] <= ingredient_rules['Upper Bound'])

  # def lower_bound_nutrients(m,o):
  #   return pyo.quicksum(m.x[o,d] for d in m.nutrients) >= nutrient_rules['Lower Bound'][o]
  # m.lower_boundnu = pyo.Contraint(m.name, rule=lower_bound_nutrients)

  # def upper_bound_nutrients(m,o):
  #   return pyo.quicksum(m.x[o,d] for d in m.nutrients) <= nutrient_rules['Upper Bound'][o]
  # m.upper_boundnu = pyo.Contraint(m.name, rule=upper_bound_nutrients)

  # def lower_bound_ingredients(m,o):
  #   return pyo(m.x[o,d] for d in m.ingredients) >= ingredient_rules['Lower Bound'][o]
  # m.lower_boundin = pyo.Contraint(m.name, rule=lower_bound_ingredients)

  # def upper_bound_ingredients(m,o):
  #   return pyo.quicksum(m.x[o,d] for d in m.ingredients) <= ingredient_rules['Upper Bound'][o]
  # m.upper_boundin = pyo.Contraint(m.name, rule=upper_bound_ingredients)
  # model.pprint()